In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import walk

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD

from keras.utils import Sequence
from keras import backend as K #for custom loss function

import tensorflow as tf #for custom loss function

In [ ]:
def mse_wrap_angle(y_true, y_pred):
    """Custom loss function based on MSE but with angles wrapped to 360 degree."""
    diff = y_pred - y_true
    if K.greater(diff,180) is not None:
        diff = diff - 360
    elif K.less(diff,-180) is not None:
        diff = diff + 360
    return K.mean(K.square(diff), axis=-1)

def mae_wrap_angle(y_true, y_pred):
    """Custom loss function based on MAE but with angles wrapped to 360 degree."""
    diff = y_pred - y_true
    if K.greater(diff,180) is not None:
        diff = diff - 360
    elif K.less(diff,-180) is not None:
        diff = diff + 360
    return K.mean(K.abs(diff), axis=-1)

def mse_wrap_angle2(y_true, y_pred):
    """
    Custom loss function based on MSE but with angles wrapped to 360 degree.
    Tensorflows atan2 is used. Therefore conversion to radiant and back is performed.
    """
    a = tf_deg2rad(y_pred)
    b = tf_deg2rad(y_true)
    diff = tf.atan2(tf.sin(a - b), tf.cos(a - b))
    diff = tf_rad2deg(diff)
    return K.mean(K.square(diff), axis=-1)

def mae_wrap_angle2(y_true, y_pred):
    """
    Custom loss function based on MAE but with angles wrapped to 360 degree.
    Tensorflows atan2 is used. Therefore conversion to radiant and back is performed.
    """
    a = tf_deg2rad(y_pred)
    b = tf_deg2rad(y_true)
    diff = tf.atan2(tf.sin(a - b), tf.cos(a - b))
    diff = tf_rad2deg(diff)
    return K.mean(K.abs(diff), axis=-1)

def tf_deg2rad(a):
    """Tensorflow compatible conversion from degree to radians."""
    return tf.divide(tf.multiply(a,np.pi),180)

def tf_rad2deg(a):
    """Tensorflow compatible conversion from degree to radians."""
    return tf.divide(tf.multiply(a,180),np.pi)

In [ ]:
def simple_model():
    """Define simple model"""
    # create model
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(96,)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # show info
    model.summary()
    # compile model
    model.compile(loss=mse_wrap_angle2,
                  optimizer=SGD(),
                  metrics = [mae_wrap_angle2])
    return model

## Training

In [ ]:
# define training parameters
batch_size = 1
num_epochs = 100

In [ ]:
# define dataset directory
data_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/' 
# load filelist
filelist = []
for (dirpath, dirnames, filenames) in walk(data_dir):
    filelist.extend(filenames)
    
# load column labels only
column_label = pd.read_csv(data_dir+filelist[1], nrows=1).columns.tolist()
feature_label = [s for s in column_label if ("ILD"  in s or "ITD" in s or "IC" in s)]
target_label = 'Localization_Azimuth'

In [ ]:
# normalization + train_test_split
data = pd.read_csv(data_dir+filelist[60])
minmax_scale = preprocessing.MinMaxScaler().fit(data[feature_label])
std_scale = preprocessing.StandardScaler().fit(data[feature_label])

train, test = train_test_split(data, test_size=0.3)
train_minmax = minmax_scale.transform(train[feature_label])
test_minmax = minmax_scale.transform(test[feature_label])
train_std = std_scale.transform(train[feature_label])
test_std = std_scale.transform(test[feature_label])

x_train = train_minmax
y_train = train[target_label]
x_test = test_minmax
y_test = test[target_label]

#data_minmax = minmax_scale.transform(data[feature_label])

In [ ]:
# model.fit on entire dataset

model = simple_model()

history = model.fit(x_train, y_train,
                    #batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

## Evaluation

In [ ]:
# evaluate model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test mae w/o wrap:', score[1])
print('Test mae w wrap:', score[2])

# plot train history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#test_data = pd.read_csv(data_dir+filelist[51])
test_features = test_data[feature_label].values
test_features_minmax = std_scale.transform(test_features)
test_targets = test_data[target_label].values


test_results = model.predict(test_features_minmax)

In [ ]:
test_results[0:100]

In [ ]:
# Save history to json file
import json
with open('file.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:
from keras.models import load_model

model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

In [ ]:
# evaluate model
score = model.evaluate(test_features, test_targets, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test accuracy:', score[2])